In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="mixtral-8x7b-32768"
)

response = llm.invoke("the first person to land on the moon was ...")
print(response.content)

Neil Armstrong, an American astronaut and aeronautical engineer, was the first person to land on the moon. He made this historic achievement on July 20, 1969, as part of the Apollo 11 mission. Upon landing, Armstrong famously declared, "That's one small step for man, one giant leap for mankind."


In [2]:
# web scrapping
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.linkedin.com/jobs/")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






 



















































































LinkedIn Job Search: Find US Jobs, Internships, Jobs Near Me

















 



      Skip to main content
    


















              Agree & Join LinkedIn
            

      By clicking Continue to join or sign in, you agree to LinkedIn’s User Agreement, Privacy Policy, and Cookie Policy.
    






















LinkedIn








                Join now
              

          Sign in
      
 

 




          Millions of jobs and people hiring
        












 





          Email or phone
        









          Password
        


Show





 

Forgot password?



          Sign in
        



              or
            





      By clicking Continue to join or sign in, you agree to LinkedIn’s User Agreement, Privacy Policy, and Cookie Policy.
    

 




            New to LinkedIn? Join now
          







          Explore topics you are interested in
      

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
{"role": "Software Engineer", "experience": "Mid-Senior Level", "skills": ["JavaScript", "React", "Node.js", "Python"], "description": "Join our team as a Software Engineer and work on building and maintaining our web applications. You will be responsible for developing new features, debugging issues, and optimizing performance. Candidates should have experience with JavaScript, React, Node.js, and Python."},
{"role": "Business Development Manager", "experience": "Senior Level", "skills": ["Sales", "Negotiation", "Market Research", "Business Strategy"], "description": "We are looking for a Business Development Manager to join our team. In this role, you will be responsible for identifying and pursuing new business opportunities, building relationships with key stakeholders, and developing strategic plans to drive growth. Candidates should have experience in sales, negotiation, market research, and business strategy."},
{"role": "Marketing Coordinator", "experience": "Entry Level", "s

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'experience': 'Mid-Senior Level',
  'skills': ['JavaScript', 'React', 'Node.js', 'Python'],
  'description': 'Join our team as a Software Engineer and work on building and maintaining our web applications. You will be responsible for developing new features, debugging issues, and optimizing performance. Candidates should have experience with JavaScript, React, Node.js, and Python.'},
 {'role': 'Business Development Manager',
  'experience': 'Senior Level',
  'skills': ['Sales', 'Negotiation', 'Market Research', 'Business Strategy'],
  'description': 'We are looking for a Business Development Manager to join our team. In this role, you will be responsible for identifying and pursuing new business opportunities, building relationships with key stakeholders, and developing strategic plans to drive growth. Candidates should have experience in sales, negotiation, market research, and business strategy.'},
 {'role': 'Marketing Coordinator',
  'experience': 'E

In [7]:
type(json_res)

list

In [8]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [10]:
links = collection.query(query_texts=["Experience in Python","Expertise in React Native"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [12]:
job = json_res  # json_res is a list of dictionaries
first_job_skills = job[0]['skills']  # Access 'skills' from the first job dictionary
print(first_job_skills)

['JavaScript', 'React', 'Node.js', 'Python']


In [14]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Excited to Help You Find the Perfect Fit for Your Team at AtliQ

Dear Hiring Manager,

I hope this email finds you well. I came across your job postings and was impressed by the opportunities available, particularly the Software Engineer, Business Development Manager, Marketing Coordinator, Data Analyst, and Product Manager roles. At AtliQ, we specialize in AI & Software Consulting and have a proven track record of delivering tailored solutions to help businesses like yours thrive.

For the Software Engineer position, our team has extensive experience in JavaScript, React, Node.js, and Python. We have showcased our expertise in these areas through projects featured in our ML Python Portfolio (<https://example.com/ml-python-portfolio>) and Python Portfolio (<https://example.com/python-portfolio>). Our engineers are well-equipped to build and maintain your web applications, develop new features, debug issues, and optimize performance.

Our Business Development Manager candidates

In [ ]:
# we are using vectordb instead of mysql and all because it is effecient in semantic searching means if we give chole bhature then it will be able to identify that we are talking about new delhi